In [994]:
%load_ext autoreload
%autoreload 2

from loader import Data, load_data_from_json
import numpy as np

import pandas as pd
from model import Model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [995]:
data: Data = load_data_from_json("../data/test.json")

In [996]:
print(data)

Data(N=3, E=2, Q=1, P=1, _qualifications=[[0], [0]], _conges=[[], [0]], _duree=[[2]], _gain=[20], _due_date=[2], _penalty=[3], _employees_names=['Olivia', 'Liam'], _qualifications_names=['A'], _projects_names=['Job1'], M=10000000.0)


In [997]:
m = Model(data)

In [998]:
m.constraint_cant_work_more_than_needed()
m.constraint_cant_work_on_days_off()
m.constraint_no_qual_work()
m.constraint_one_project_per_day()

In [999]:
m.determine_affecte()
m.determine_debute()
m.determine_realise()

In [1000]:
m.objective_max_gain()

In [1001]:
m.update()

In [1002]:
m.optimize()


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 4700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 14 columns and 51 nonzeros
Model fingerprint: 0x48e9fd8d
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 16 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 20 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%


In [1003]:
def get_completion_day(p):
    for j in range(m.data.N):
        if(m.realise(p, j).X == 1):
            return j
    return None

def get_work(p):
    df = pd.DataFrame()
    for e in range(m.data.E):
        work_done = []
        for j in range(m.data.N):
            work = []
            for q in range(m.data.Q):
                work.append(m.projet(e, j, p, q).X)
            has_worked = 1 in work
            if(not has_worked):
                if j in m.data.conges(e):
                    work_done.append("X")
                else:
                    work_done.append(" ")
            else:
                work = np.argmax(work)
                work = m.data.qualification_name(work)
                work_done.append(work)
        df[m.data.employee_name(e)] = work_done

    due = []
    for j in range(m.data.N):
        if(j == m.data.due_date(p)): due.append("X")
        else: due.append("")
    
    completed = []
    completion_day = get_completion_day(p)
    for j in range(m.data.N):
        if(j == completion_day): completed.append("X")
        else: completed.append("")

    df["Due_Date"] = due
    df["Completed"] = completed
    
    return df
    

In [1004]:
for p in range(m.data.P):
    qualis = []
    for q in range(m.data.Q):
        qualis.append((m.data.qualification_name(q), m.data.duree(p, q)))
    print("Work on", m.data.projet_name(p), qualis)
    print(get_work(p))
    completion_day = get_completion_day(p)
    if(completion_day != None):
        print("Project completed, gained", m.data.gain(p))
        if(m.data.due_date(p) < completion_day):
            print("Project late", completion_day - m.data.due_date(p), "day, lost", m.data.penalite(p, completion_day))
        print("Total gain", m.data.gain(p) - m.data.penalite(p, completion_day))
    else:
        print("Project not completed")
        print("Project late", m.data.N - 1 - m.data.due_date(p), "day, lost", m.data.penalite(p, m.data.N - 1))
        print("Total gain", -m.data.penalite(p, m.data.N - 1))

    print()



Work on Job1 [('A', 2)]
  Olivia Liam Due_Date Completed
0      A    X                   
1      A                        
2                    X         X
Project completed, gained 20
Total gain 20



In [1005]:
# print(m._realise)

# print(m.data.duree(0, 0))
# for j in range(2):
#     print(m.projet(0,j,0,0))

# print(m.realise(0, 1))

j = 1

sum_duree = m.data.duree(p, q)
sum_projet = sum(
    m.projet(0, j_prime, 0, 0).X
    for j_prime in range(j + 1)
)

print(sum_duree, sum_projet, m.realise(0, j).X)


2 2.0 0.0


In [1006]:
# 2 - 2 <= (1 - 0)